## 2020 Israel Elections, Numerical Observations

In [1]:
import pandas as pd
import numpy as np

# read counted votes by city
df = pd.read_csv("https://media23.bechirot.gov.il/files/expc.csv", encoding='iso8859_8')

# establish list of parties
parties = df.columns[7:-1]

# sum votes and sort
top10 = df[parties].sum().sort_values(ascending=False).head(10)
top10

# extract call letters of top parties
call10 = top10.index.values
call10

df.head(2)

,סמל ועדה,שם ישוב,סמל ישוב,בזב,מצביעים,פסולים,כשרים,אמת,ג,ודעם,...,נק,פה,ףז,ץ,ק,קי,קך,קץ,שס,Unnamed: 37
0,5,כפר כנא,509,682,488,3,485,2,0,480,...,0,1,0,0,0,0,0,0,0,NaN
1,17,אחווה,1157,185,143,0,143,13,0,0,...,0,88,0,0,0,0,0,0,2,NaN


### OB1 - Places best fitting national voting pattern

- Which are the places the vote closest to the national average?
- What does it say about such places?
- Do they show a special kind of mix? living environment?
- Interview a person from such a place. Does he/she feels it?

In [5]:
# add column r1 for ratio of smillarity to overall results
# nature of r1 is still under question (how to compute)

he_valid = 'כשרים'
he_voters = 'מצביעים'
he_place = 'שם ישוב'

top10r = top10.divide(df[he_valid].sum())
def calcFit(r):
    return(r.filter(items=call10).divide(r[he_valid]) - top10r).abs().sum()

df['fit_index'] = df.apply(lambda x: calcFit(x), axis=1)
df_sig_places = df[df[he_voters]>5000]
df_sel_cols = df_sig_places[np.concatenate([[he_place, he_voters, he_valid, 'fit_index'], call10])]
df_sel_cols.sort_values(by=['fit_index'], ascending=True).head(10)

,שם ישוב,מצביעים,כשרים,fit_index,מחל,פה,ודעם,שס,ג,אמת,ל,טב,נץ,ז
1214,מעטפות חיצוניות,305452,303187,0.224014,98952,89856,16649,21945,12742,22223,11829,25398,1600,578
217,רחובות,74296,73986,0.279370,25292,24095,257,5644,3283,4854,5204,4644,443,71
178,חיפה,145488,144625,0.280597,38464,50081,14915,4218,5604,10395,14893,5020,356,154
208,פתח תקווה,131620,130974,0.313459,47074,37258,190,10787,6339,5022,11412,11849,558,118
578,ירכא,6444,6325,0.335600,1860,1993,997,305,0,205,904,19,2,0
224,אזור,6730,6689,0.413788,2750,2148,24,739,76,339,317,231,33,2
270,חדרה,48251,47954,0.417133,21607,13046,214,4011,667,1887,4648,1498,180,36
350,ראש העין,33310,33216,0.421804,13336,11658,43,3146,740,1325,1081,1627,200,13
177,חולון,101584,101066,0.425617,42980,31303,163,10345,894,4606,7081,2914,364,92
695,חריש,5160,5135,0.426728,1669,1121,68,758,159,175,324,768,64,9


### OB2 - Places noteworthy for throwing votes to dumpster

- Which are the places where highest percentage of voters voted for parties that did not make it into the parliment?
- What does it say about such a place?
- Is it small? big? are they concentrated around specific areas?
- Is the order by magnitude (rather than percentage) radically different?

### OB3 - The lone voters

- Are there places where there was only one voter (or a very small number, 2?) that voted for a maor party
- How must it feel?
- Can he he/she be located?


### OB4 - Case study: Netz

- for the party of נץ - which was certain to not make it ...
- What can explain the large number of voters?
- Where are they?
- Do they define a "resistence"? hard-core? ??
- Interview

### OB5 - Corona virus voters?

- Is data available for the "isolation" voting booths?
- Is the pattern similar to national
- What is the percentage of voters?
- Interview